In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''') 

# <font color="#E56717">Comparative Effective Research</font>
## <font color="steelblue">Using SEER-Medicare to compare the efficacy and cost-effectiveness of different radiation treatment modalities for prostate cancer patients</font>

## IDA, 21 April 2017

### David Dooling

In [2]:
from IPython.display import IFrame

# <font color="#E56717">The Problem:</font>

* <span style="font-size:1.3em;">Two main classes of different types of external radiation beam therapy: $\gamma$ (photons) or other actual particles (protons in this case, but also research underway using carbon ions)</span>
* <span style="font-size:1.3em;">A $\gamma$-beam relatively inexpensive to generate, but less **controllable**. State of the art is $\bf{I}$ntensity $\bf{M}$odulated $\bf{R}$adiation $\bf{T}$herapy, but still causes collateral damage</span>
* <span style="font-size:1.3em;">$\bf{P}$roton $\bf{B}$eam $\bf{T}$herapy: More expensive to generate the particle beam, but causes much less collateral damage, based on plain physics and engineering principles</span>

###  <font color="red">Insurance companies pushing back; denying claims for PBT. Complicated appeal processes. Lack of clinical "evidence" given as justification. No RCT supporting the assumed benefits of PBT</font>

In [12]:
IFrame('http://www.bbc.com/news/uk-england-hampshire-29106294',width=800,height=400) # emotional appeals are moving, but will not sway insurance companies

## <font color="#E56717">Emotional appeals will not sway insurance companies. No RCT to point to; most practitioners will not knowingly give one arm of the study what they feel to be inferior treatment.</font>

# <font color="steelblue">Observational studies: Use the SEER-Medicare database</font>

# <font color="#E56717">Identify all the patients in the SEER-MEdicare data who have received PBT</font>

<span style="font-size:1.7em;">Can be done via $\bf{H}$ealthcare $\bf{C}$ommon $\bf{P}$ocedure $\bf{C}$odes:</span>

* <span style="font-size:1.7em;">75520 - Proton trmt simple w/o comp</span>
* <span style="font-size:1.7em;">77522 - Proton trmt simple w/comp</span>
* <span style="font-size:1.7em;">77523 - Proton trmt intermediate</span>
* <span style="font-size:1.7em;">77525 - Proton treatment complex</span>

# <font color="#E56717">Likewise identify all patients who have received IMRT</font>

* <span style="font-size:1.7em;">77418 - Intensity modulated treatment delivery</span>